In [33]:
import pandas as pd
import numpy as np
import os
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from time import time
from IPython.display import clear_output
from keras .preprocessing.text import one_hot, Tokenizer
from keras.utils import pad_sequences
import nltk
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, Input
from keras.models import Model
import keras
import json

In [34]:
dataDir = os.listdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data")
os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data")

In [35]:
def check(token):
    for i in range(len(token)):
        if ord(token[i]) not in range(97, 123):
            return False
    return True

In [36]:
corpus = []
for file in os.listdir("OCR"):
    df = pd.read_csv("OCR/" + file)
    text = df["text"].to_numpy()
    tokens = []
    for t in text:
        T = t.lower()
        while len(T) >0 and ord(T[0]) not in range(97, 123):
            T = T[1:]
        while len(T)>0 and ord(T[-1]) not in range(97, 123):
            T = T[:-1]
        if len(T)>1 and check(T):
            tokens.append(T)
    T = " ".join(tokens)
    corpus.append(T)



In [37]:
text = " ".join(corpus)

In [38]:
tokens = text.split(" ")
len(tokens)

18039

In [39]:
valid = []
for t in tokens:
    if t not in valid:
        valid.append(t)

In [40]:
len(valid)

3551

In [41]:
len(corpus)

110

In [42]:
vocab_size = 4000
encoded_corp = []
for i, doc in enumerate(corpus):
    encoded_corp.append(one_hot(doc, vocab_size))
    print("Encoding for doc ", i+1, " is : ", one_hot(doc, vocab_size))


Encoding for doc  1  is :  [3829, 1285, 699, 3817, 1031, 168, 2993, 168, 690, 3949, 3829, 168, 79, 2737, 3125, 3065, 603, 2880, 3441, 1074, 3282, 1888, 708, 2954, 3410, 838, 722, 1935, 2320, 569, 998, 1125, 1353, 2404, 1125, 1353, 2920, 3833, 3308, 806, 843, 1091, 325, 3133, 843, 1091, 2784]
Encoding for doc  2  is :  [969, 3829, 3503, 3829, 418, 1980, 3814, 2392, 1940, 1562, 235, 673, 3696, 1695, 596, 2993, 2872, 810, 2258, 2991, 2651, 51, 621, 3983, 418, 168, 2905, 63, 621, 235, 278, 1207, 2880, 699, 418, 1980, 2308, 2538, 2081, 31, 1551, 3178, 841, 1263, 3052, 722, 2709, 1307, 418, 2993, 2872, 168, 1935, 2320, 528, 228, 2228, 2037, 2991, 2651, 1770, 621, 1318, 810, 2993, 2700, 1642, 810, 2627, 937, 1586, 3123, 3431, 418, 972, 1462, 1455, 169, 1173, 3571, 2245, 3747, 2790, 3431, 2022, 3375, 1039, 1185, 1078, 3799, 661, 3799, 3774, 2937, 3713, 700, 2154, 223, 2100, 1801, 3058, 3921, 969, 2790, 1544, 3749, 1586, 3431, 969, 3431, 2790, 969, 3431, 3375, 1039, 1185, 223, 661, 1500, 3774, 

In [43]:
maxlen = -1
for doc in corpus:
    tokens = nltk.word_tokenize(doc)
    if maxlen<len(tokens):
        maxlen = len(tokens)
maxlen

426

In [44]:
pad_corpus = pad_sequences(encoded_corp, maxlen=maxlen, padding="post", value=0)

In [45]:
OD = 1

In [46]:
input = Input(shape=(len(corpus), maxlen), dtype="float64")

In [47]:
word_input = Input(shape=(maxlen, ), dtype="float64")

word_embeddings = Embedding(input_dim=vocab_size, output_dim=OD, input_length=maxlen)(word_input)

word_vec = Flatten()(word_embeddings)

embed_model = Model([word_input], word_vec)

In [48]:
embed_model.compile(optimizer=keras.optimizers.Adam(), loss="binary_crossentropy", metrics=["acc"])

In [49]:
embed_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 426)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 426, 1)            4000      
                                                                 
 flatten_1 (Flatten)         (None, 426)               0         
                                                                 
Total params: 4,000
Trainable params: 4,000
Non-trainable params: 0
_________________________________________________________________


In [50]:
embeddings = embed_model.predict(pad_corpus)

4/4 [==============================] - 0s 1ms/step


In [51]:
embeddings.shape

(110, 426)

In [52]:
embeddings

array([[-0.03879399, -0.00578324,  0.03113684, ..., -0.00445103,
        -0.00445103, -0.00445103],
       [-0.04227363, -0.03879399, -0.04014883, ..., -0.00445103,
        -0.00445103, -0.00445103],
       [ 0.01379485, -0.00777174, -0.00777174, ..., -0.00445103,
        -0.00445103, -0.00445103],
       ...,
       [ 0.01393154, -0.03318512,  0.04103087, ..., -0.00445103,
        -0.00445103, -0.00445103],
       [-0.03879399, -0.02156727, -0.04122981, ..., -0.00445103,
        -0.00445103, -0.00445103],
       [ 0.01625529, -0.04014883, -0.04222177, ..., -0.00445103,
        -0.00445103, -0.00445103]], dtype=float32)

In [53]:
embeddings = embeddings.reshape(-1, maxlen, OD)
embeddings

array([[[-0.03879399],
        [-0.00578324],
        [ 0.03113684],
        ...,
        [-0.00445103],
        [-0.00445103],
        [-0.00445103]],

       [[-0.04227363],
        [-0.03879399],
        [-0.04014883],
        ...,
        [-0.00445103],
        [-0.00445103],
        [-0.00445103]],

       [[ 0.01379485],
        [-0.00777174],
        [-0.00777174],
        ...,
        [-0.00445103],
        [-0.00445103],
        [-0.00445103]],

       ...,

       [[ 0.01393154],
        [-0.03318512],
        [ 0.04103087],
        ...,
        [-0.00445103],
        [-0.00445103],
        [-0.00445103]],

       [[-0.03879399],
        [-0.02156727],
        [-0.04122981],
        ...,
        [-0.00445103],
        [-0.00445103],
        [-0.00445103]],

       [[ 0.01625529],
        [-0.04014883],
        [-0.04222177],
        ...,
        [-0.00445103],
        [-0.00445103],
        [-0.00445103]]], dtype=float32)

In [54]:
dict = {}
for i in range(len(corpus)):
    doc = corpus[i].split(" ")
    for j in range(len(doc)):
        dict[doc[j]] = embeddings[i][j]


In [55]:
from sklearn.metrics.pairwise import cosine_similarity

In [56]:
print(cosine_similarity([dict["date"]], [dict["dated"]]))

[[-1.]]


In [57]:
dict["invoice"][0]

-0.038793992

In [58]:
for key in dict.keys():
    val = dict[key][0]
    val = float(val)
    dict[key] = val
dict

{'invoice': -0.03879399225115776,
 'pitstop': -0.005783237516880035,
 'order': 0.031136836856603622,
 'id': 0.010334979742765427,
 'service': 0.0252956785261631,
 'date': -0.03593393415212631,
 'delivery': -0.021567273885011673,
 'registration': 0.008700598031282425,
 'number': 0.035023752599954605,
 'renault': -0.04446839168667793,
 'pulse': -0.019214725121855736,
 'diesel': -0.04597220569849014,
 'odometer': 0.02414466068148613,
 'from': 0.03355500474572182,
 'to': 0.03222217783331871,
 'cars': 0.03802556172013283,
 'care': -0.02098955027759075,
 'opp': -0.007129956036806107,
 'trident': 0.01780029758810997,
 'hyundai': 0.03034130111336708,
 'showroom': -0.028101205825805664,
 'kudlu': 0.02260586991906166,
 'gate': -0.026707543060183525,
 'hosur': 0.03071046993136406,
 'main': -0.049713920801877975,
 'road': 0.008328437805175781,
 'bengaluru': -0.040259670466184616,
 'gst': 0.04365856572985649,
 'bumper': 0.03016388788819313,
 'bracket': 0.021109189838171005,
 'front': -0.01892548799

In [59]:
pwd

"/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data"

In [61]:
os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Models")

In [62]:
with open("wordEmbeddings.json", "w") as file:
    json.dump(dict, file)

In [63]:
len(dict)

3551